In [1]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /kaggle/working/processed_data.zip
100%|██████████████████████████████████████| 6.00M/6.00M [00:00<00:00, 66.7MB/s]


In [2]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [3]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

In [4]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [5]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [6]:


models_list_3=[
        "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
"FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",

# "bert",

]


In [7]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
# langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [8]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




not found: processed_data/train/track_a/ind.csv
not found: processed_data/train/track_a/jav.csv
not found: processed_data/train/track_a/xho.csv
not found: processed_data/train/track_a/zul.csv


In [9]:
all_csv

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...
61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [10]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [11]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [12]:
train_data=all_csv


In [13]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [14]:

train_data=train_data.fillna(0)

train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...
61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [15]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...
61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [16]:
train_data.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [17]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...,...
61103,61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [18]:
train_data

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,amh_train_track_a_00001,1,1,0,0,0,0,amh,እናንተ መቸም አትማሩም ምድረ አውሬ ሁላ
1,amh_train_track_a_00002,1,1,0,0,0,0,amh,ምነው የዶክተር እስራኤል ጥላሁን ሞት በዝምታ አለፋችሁት ባለስልጣን ስለሆ...
2,amh_train_track_a_00003,0,0,0,0,0,0,amh,እንዲሁም ይህ እጅግ እርሶን የሚወዶት እና እጅግም የሚሳሳልዎን የአዲስአበ...
3,amh_train_track_a_00004,1,1,0,0,0,0,amh,etv ዘረኛ ናቹሁ አሽቃባጭ ስንት አባቶቻችን መስዋት የሆኑበትን ባዲራ እ...
4,amh_train_track_a_00005,0,1,0,1,0,0,amh,የዛሬው የጦጣ ንክሻ ይለያል ክክ አይ ፊሊፖስ አንተ ጅማታም እንድህ ቅጥል...
...,...,...,...,...,...,...,...,...,...
61103,yor_train_track_a_02988,0,0,0,0,0,0,yor,oju ti pọn awọn fulani ju ni naijiria yii afi ...
61104,yor_train_track_a_02989,1,0,0,0,0,0,yor,á júwe ilé fún niyi akintola tó bá tẹ̀síwájú l...
61105,yor_train_track_a_02990,0,0,0,0,0,0,yor,nigeria vs madagascar super eagles ń gbáradì f...
61106,yor_train_track_a_02991,0,0,0,0,0,0,yor,tb joshua biography tí kìí bá ṣe rélùwéè tó bà...


In [19]:
# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise'] #eng
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise'] #all
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness',]#afr


In [20]:
train_data['lang'].value_counts()

lang
pcm     3728
tir     3681
amh     3549
orm     3442
som     3392
swa     3307
yor     2992
ibo     2880
rus     2679
chn     2642
deu     2603
hin     2556
ukr     2466
kin     2451
mar     2415
ptbr    2226
hau     2145
esp     1996
ary     1608
vmw     1551
ptmz    1546
ron     1241
swe     1187
tat     1000
sun      924
arq      901
Name: count, dtype: int64

In [21]:
sum(train_data['lang'].isna())

0

In [22]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [23]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [24]:
train_data[train_data["lang"]=="deu"].dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [25]:
nan_rows["lang"].value_counts()

Series([], Name: count, dtype: int64)

In [26]:
nan_rows.dtypes

id               object
anger             int64
disgust           int64
fear              int64
joy               int64
sadness           int64
surprise          int64
lang             object
clean_message    object
dtype: object

In [27]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

               id  disgust   fear    joy  sadness  surprise   lang  \
lang                                                                 
amh  0      False    False  False  False    False     False  False   
     1      False    False  False  False    False     False  False   
     2      False    False  False  False    False     False  False   
     3      False    False  False  False    False     False  False   
     4      False    False  False  False    False     False  False   
...           ...      ...    ...    ...      ...       ...    ...   
yor  61103  False    False  False  False    False     False  False   
     61104  False    False  False  False    False     False  False   
     61105  False    False  False  False    False     False  False   
     61106  False    False  False  False    False     False  False   
     61107  False    False  False  False    False     False  False   

            clean_message  
lang                       
amh  0              False  
     

In [28]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [29]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [30]:
x_train, y_train = train['clean_message'].values.tolist(), train[label_columns].values.tolist()

x_valid, y_valid = valid['clean_message'].values.tolist(), valid[label_columns].values.tolist()

In [31]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [32]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1    sum
anger       39986     8900  48886
disgust     40290     8596  48886
fear        44848     4038  48886
joy         39316     9570  48886
sadness     39863     9023  48886
surprise    43397     5489  48886


In [33]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [34]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([2.7464, 2.8435, 6.0532, 2.5541, 2.7090, 4.4531], device='cuda:0')

In [35]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [36]:
!ls output/best-model-v1

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [37]:
# def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

#   train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

#   valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

#   train_dataset = Dataset(train_encodings, y_train)

#   valid_dataset = Dataset(valid_encodings, y_valid)

#   return train_dataset,valid_dataset


def build_dataset(x_train, y_train, x_valid, y_valid, MAX_LENGTH, tokenizer):
    assert isinstance(x_train, list) and isinstance(x_valid, list), "x_train and x_valid must be lists of strings"
    train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH, return_tensors="pt")
    valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH, return_tensors="pt")
    train_dataset = Dataset(train_encodings, y_train)
    valid_dataset = Dataset(valid_encodings, y_valid)
    return train_dataset, valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=100,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [38]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)
    print("Labels shape:", eval_pred.label_ids.shape)
    print("Predictions shape:", eval_pred.predictions.shape)


    labels = np.asarray(labels)
    predictions = np.asarray(predictions)
    
    assert labels.shape == predictions.shape, f"Shape mismatch: {labels.shape} vs {predictions.shape}"

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [39]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average=None):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    
    # probabilities = expit(logits)
    logits = model(**inputs).logits
    probabilities = expit(logits.cpu().numpy()) 
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [40]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [41]:
MAX_LENGTH = 256

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [42]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [43]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [44]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [45]:
USE_LORA=False

In [46]:
from transformers import AutoModel

In [47]:
# quantize=True
quantize=False

In [48]:


torch.cuda.empty_cache()



print(x)
tokenizer = AutoTokenizer.from_pretrained(x)
embedding_model = AutoModel.from_pretrained(x, output_hidden_states=True,quantization_config=quantization_config if quantize else None)

FacebookAI/xlm-roberta-large


In [49]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [50]:
# a=

In [51]:

# Define the MLP Classifier
class CustomClassifier(nn.Module):
    def __init__(self, input_size, num_classes, kernel_size=3, num_filters=256):
        super(CustomClassifier, self).__init__()
        self.conv = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        self.fc = nn.Linear(num_filters, num_classes)
        self.dropout = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x):
        conv_out = F.relu(self.conv(x.float()))
        pooled_conv_out, _ = torch.max(conv_out, dim=2)
        pooled_conv_out=self.dropout2(pooled_conv_out)
        logits = self.fc(pooled_conv_out)
        return logits


In [52]:
# # Define a custom loss function to handle class weights for multilabel classification
# def custom_loss(output, target):
#     # BCEWithLogitsLoss with pos_weight for each class
#     loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)
#     return loss_fn(output, target)


def custom_loss(output, target):
    # BCEWithLogitsLoss with pos_weight for each class
    loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)
    return loss_fn(output, target)

In [53]:

# Combine XLM-RoBERTa with MLP
class XLMRobertaWithMLP0(nn.Module):
    def __init__(self, embedding_model, classifier):
        super(XLMRobertaWithMLP, self).__init__()
        self.embedding_model = embedding_model
        for param in self.embedding_model.parameters():
            param.requires_grad = False
        self.classifier = classifier

    def forward(self, input_ids, attention_mask, labels=None):
        # Get embeddings from XLM-RoBERTa
        output = self.embedding_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output if output.pooler_output is not None else output.last_hidden_state.mean(dim=1)
        pooled_output = pooled_output.unsqueeze(1)
        pooled_output = pooled_output.repeat(1, 1024, 1)
        # Pass embeddings to MLP
        logits=self.classifier(pooled_output)
        if labels is not None:
            loss = custom_loss(logits, labels.float())
            return logits, loss
        return logits


In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the updated CustomClassifier
class CustomClassifier1(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CustomClassifier1, self).__init__()
        
        # Dropout layers
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.2)

        # Bidirectional GRU layers
        self.gru1 = nn.GRU(input_size, 120, batch_first=True, bidirectional=True)  # Input size matches embedding size
        self.gru2 = nn.GRU(240, 64, batch_first=True, bidirectional=True)  # Output size is 128 (64*2)
        self.gru3 = nn.GRU(128, 64, batch_first=True, bidirectional=True)  # Output size is 128 (64*2)

        # Batch normalization
        self.batch_norm = nn.BatchNorm1d(128)  # Match input_size to GRU2 output (64*2)

        # Fully connected output layer
        self.fc = nn.Linear(128, num_classes)  # Match GRU3 output (64*2)

    def forward(self, x):
        # Apply first dropout
        x = self.dropout1(x)

        # GRU layers
        gru_out1, _ = self.gru1(x)  # Output shape: (batch, seq_len, 120*2)
        gru_out2, _ = self.gru2(gru_out1)  # Output shape: (batch, seq_len, 64*2)

        # Batch normalization
        normalized = self.batch_norm(gru_out2.transpose(1, 2)).transpose(1, 2)

        # Final GRU layer
        gru_out3, _ = self.gru3(normalized)  # Output shape: (batch, seq_len, 64*2)

        # Average pooling
        pooled_output = torch.mean(gru_out3, dim=1)  # Shape: (batch, 64*2)

        # Final dense layer
        logits = self.fc(pooled_output)  # Shape: (batch, num_classes)
        return logits



# Combine XLM-RoBERTa with the updated classifier
class XLMRobertaWithMLP1(nn.Module):
    def __init__(self, embedding_model, classifier):
        super(XLMRobertaWithMLP1, self).__init__()
        self.embedding_model = embedding_model
        for param in self.embedding_model.parameters():
            param.requires_grad = False
        self.classifier = classifier

    def forward(self, input_ids, attention_mask, labels=None):
        # Get embeddings from XLM-RoBERTa
        output = self.embedding_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output if output.pooler_output is not None else output.last_hidden_state.mean(dim=1)

        # Reshape the pooled output for the classifier
        pooled_output = pooled_output.unsqueeze(1).repeat(1, 50, 1)  # Adjust sequence length (50) as needed

        # Pass embeddings to the classifier
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss = custom_loss(logits, labels.float())
            return logits, loss
        return logits


In [55]:

# Define your MLP with appropriate dimensions
hidden_size = 1024  # For XLM-RoBERTa-large, hidden size = 1024
classifier = CustomClassifier1(input_size=hidden_size, num_classes=len(label_columns))

# Combine the models

# model = XLMRobertaWithMLP0(embedding_model=embedding_model, classifier=classifier)

model = XLMRobertaWithMLP1(embedding_model=embedding_model, classifier=classifier)


In [56]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [57]:
if tokenizer.pad_token is None:
    print("adding tokenizer token")
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer.pad_token=tokenizer.eos_token

    embedding_model.config.pad_token_id = tokenizer.pad_token_id

    # model.resize_token_embeddings(len(tokenizer))

    if embedding_model.config.pad_token_id is None:
        print("setting model pad token")
        embedding_model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.pad_token

'<pad>'

In [58]:
device

'cuda'

In [59]:
model = model.to(device)

for param in model.parameters():
    param.data = param.data.contiguous()


In [60]:
os.environ['WANDB_DISABLED']="true"
training_args = train_arg()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [61]:
x_train = [str(text) for text in x_train]
x_valid = [str(text) for text in x_valid]


In [62]:
train_dataset,valid_dataset=build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH,tokenizer)

In [63]:
# Define the Trainer with custom loss function
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
        labels = inputs.get("labels")
        logits,loss = model(**inputs)
        loss = custom_loss(logits, labels)
        return (loss, logits) if return_outputs else loss

In [64]:
trainer = CustomTrainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    eval_dataset=valid_dataset,

    compute_metrics=compute_metrics,

    # data_collator=data_collator

)

In [65]:
checkpoints = sorted(glob.glob(os.path.join(trainer.args.output_dir, "checkpoint-*")), key=os.path.getmtime)



resume_checkpoint = True if checkpoints else False

embedding_model.config.use_cache = True
# embedding_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train(resume_from_checkpoint=resume_checkpoint)

# embedding_model.config.use_cache = True


<ipython-input-35-c02315c193cc>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss


Labels shape: (12222, 6)
Predictions shape: (11458, 6)


AssertionError: Shape mismatch: (12222, 6) vs (11458, 6)

In [ ]:
metrics,predicted_classes = evaluate_model(
  model=model,
  valid_dataset=valid_dataset,
  compute_metrics=compute_metrics,
  batch_size=32,
  device=device
)
accuracy=metrics["accuracy"]
precision=metrics["precision"]
recall=metrics["recall"]
f1=metrics["f1"]


new_data = pd.DataFrame({

  "Models_name": [x],

  "Precision": [precision],

  "Recall": [recall],

  "F1_score": [f1],

  "Accuracy": [accuracy]

})



metrics_df = pd.concat([metrics_df, new_data], ignore_index=True)



model_confusion_matrices = []








if isinstance(predicted_classes, tuple) :

  predicted_classes = predicted_classes[0]

predicted_classes = np.array(predicted_classes)

# print(f"Type after conversion: {type(predicted_classes)}")



if len(predicted_classes.shape) > 2:

  predicted_classes = predicted_classes.reshape(predicted_classes.shape[0], -1)

  # print(f"Shape after reshaping: {predicted_classes.shape}")









predicted_classes = (expit(predicted_classes) >= 0.5).astype(int)



y_valid_np = np.array(y_valid)



for j in range(len(label_columns)):

    print(f"Computing confusion matrix for label: {label_columns[j]}")

    cm = confusion_matrix(y_valid_np[:, j], predicted_classes[:, j])



    # cm = confusion_matrix(y_valid[:, j], predicted_classes[:, j])

    model_confusion_matrices.append(cm)

confusion_matrices.append(model_confusion_matrices)





print(f"{x} evaluates:")

for k, label in enumerate(label_columns):

  print(f"Class '{label}': Precision: {precision[k]:.4f}, Recall: {recall[k]:.4f}, F1: {f1[k]:.4f}")

print(f"Overall accuracy: {accuracy:.4f}")

print("-------------------------------------------------------------------------\n")

In [ ]:
# Initialize a DataFrame to store the evaluation results for each language
lang_metrics_df = pd.DataFrame(columns=["Language", "Precision", "Recall", "F1_score", "Accuracy"])
# Temporarily disable metric computation
original_compute_metrics = trainer.compute_metrics
trainer.compute_metrics = None

# Iterate over each language
for lang in langs:
    # Filter the validation data for the current language
    lang_valid_data = valid[valid['lang'] == lang]
    lang_x_valid = lang_valid_data['clean_message'].values.tolist()
    lang_y_valid = lang_valid_data[label_columns].values.tolist()
    if(len(lang_valid_data)==0):
      continue
    # Tokenize the validation data
    lang_valid_encodings = tokenizer(lang_x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)
    lang_valid_dataset = Dataset(lang_valid_encodings, lang_y_valid)

    metrics,_ = evaluate_model(
      model=model,
      valid_dataset=lang_valid_dataset,
      compute_metrics=compute_metrics,
      batch_size=32,
      device=device,
      average='weighted'
    )
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    # Store the metrics in the DataFrame
    new_data = pd.DataFrame({
        "Language": [lang],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
        "Accuracy": [accuracy]
    })

    lang_metrics_df = pd.concat([lang_metrics_df, new_data], ignore_index=True)

# Display the evaluation results for each language
print(lang_metrics_df)

# Restore compute_metrics function if needed
trainer.compute_metrics = original_compute_metrics

In [ ]:
a=

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = token

In [ ]:
trainer.push_to_hub("roberta-sentiment")